# Python Function Malarkey Ignition

### 2020-04-17

## Contents

* [Introduction](#intro)
* [Prerequisite: Mutable and Immutable Objects](#prereq)
* [Function Arguments](#arguments)
* [Default Arguments](#default-arguments)
* [Namespaces & Scopes](#scopes)
* [Closures](#closures)
* [Decorators](#decorators)
    * [Simple Decorators](#simple-decorators)
    * [Parametrized Decorators](#parametrized-decorators)
* [Concluding Thoughts](#conclusions)
* [References](#references)


**Note:** This notebook only runs in Python 3.5 or above.

## Introduction <a class="anchor" id="intro"></a>

The goal of this Ignition is to introduce and expand some concepts related to functions in Python.

## Prerrequisite: Mutable and Immutable Objects <a class="anchor" id="prereq"></a>

Do you like Object Oriented Programming? I hope so, because _everything_ in Pyhon is an object. Repeat after me:

> Everything in Pytyhon is an object.

So... buckle up.

The thing about objects is that there are multiple kinds of objects. For our purposes we'll consider only one: mutable vs. immutable.

Consider the following objects:

In [1]:
object_1 = 32423432  # integer object (immutable)
object_2 = [32423432]  # list object (mutable)

Without looking too much under the hood, when we ran the previous cell, Python allocated some memory to save the object named `object_1`, and some other memory to save `object_2`. We can consult the **memory address** (exactly where something is stored) using the `id` built-in method.

In [2]:
# Outputs will vary each session.
print(f'object_1 is stored at {id(object_1)}.')
print(f'object_2 is stored at {id(object_2)}.')

object_1 is stored at 140216130896720.
object_2 is stored at 140216130834248.


**Mutability** is an attribute of an object in Python that indicates whether modifying it changes the memory address or not. For example, integers (like `object_1`) are immutable, whereas lists are mutable. Let's see this in action.

In [3]:
# For object 1
# See the original address as above
print(f'As before, object_1 is stored at {id(object_1)}.')
# Change the object
object_1 = object_1 + 1
# Check memory address after the change.
print(f'After adding 1, object_1 is stored at {id(object_1)}.')
# Repeat with a different syntax
object_1 += 1
print(f'Note that the change is independent of syntax, since object_1 is now stored at {id(object_1)}.')

print('')

# For object_2
# See the original address as above
print(f'As before, object_2 is stored at {id(object_2)}.')
# Change the object
object_2.append(1)
# Check memory address after the change.
print(f'After appending 1 to the list, object_2 is stored at {id(object_2)}.')

As before, object_1 is stored at 140216130896720.
After adding 1, object_1 is stored at 140216130896656.
Note that the change is independent of syntax, since object_1 is now stored at 140216130896720.

As before, object_2 is stored at 140216130834248.
After appending 1 to the list, object_2 is stored at 140216130834248.


A final note on mutability: We mentioned that lists are mutable, which is true insofar as you're looking at the list itself. Objects within a list can be either mutable or immutable. For example, `object_2` is a list (mutable) that contains two integers (immutable). This means that when we try to modify the list itself by adding or removing elements, reordering, etc. we'll keep the memory address. When we try to modify an immutable element of the list, the list will retain its address and the element's will change,

In [4]:
print(f'Remember that object_2 is stored at {id(object_2)}.')

print(f'The first element of object_2 is stored at {id(object_2[0])}.')
# Change the first item
object_2[0] += 1
# Check memory addresses after the change.
print(f'After adding 1 to the first element of object_2, object 2 remains stored at {id(object_2)}, ')
print(f'and its first element is now stored at {id(object_2[0])}.')

Remember that object_2 is stored at 140216130834248.
The first element of object_2 is stored at 140216130896688.
After adding 1 to the first element of object_2, object 2 remains stored at 140216130834248, 
and its first element is now stored at 140216130896944.


## Function Arguments <a class="anchor" id="arguments"></a>

In Python, a **function parameter** is a variable name for inputs that it receives, and a **function argument** is the value it's passed. For example the following function has the parameter `a`, and is called on using the argument `7`:

In [5]:
def my_func(a):
    return a + 1

my_func(7)

8

This distinction is merely academic and no one really cares, using both terms interchangeably. The reason I'll make this distinction for now is that I believe that using a more precise language will help you grok what we'll cover in this section.

There are two ways Python functions can receive arguments:

1. Positionally: The position of the argument determines to which parameter it gets assigned.
2. By keywords: The parameter-argument is explicitly specified.

In [6]:
def my_func(a, b, c):
    """
    A function with 3 parameters.
    """
    return a, b, c

print(f'Running the function passing all arguments by position: {my_func(1, 2, 3)}')
print(f'Running the function passing all arguments by keyword: {my_func(a=1, b=2, c=3)}')
print(f'Running the function mixing methods: {my_func(1, b=2, c=3)}')

Running the function passing all arguments by position: (1, 2, 3)
Running the function passing all arguments by keyword: (1, 2, 3)
Running the function mixing methods: (1, 2, 3)


Python allows you to define functions that take an arbitrary number of arguments as follows:

In [7]:
def my_func(a, b, *args):
    """
    Function that can receive an arbitrary number of arguments.
    """
    # a, b work the same.
    print(a, b)
    # all other arguments get compacted into the tuple called args.
    print(args)
    return None

my_func(1, 2, 3, 4, 5, 6)

1 2
(3, 4, 5, 6)


Note that since Python doesn't allow you to use the positional method after you used the keyword method, you _can't_ mix methods:

In [8]:
my_func(a=1, b=2, 3, 4, 5, 6)

SyntaxError: positional argument follows keyword argument (<ipython-input-8-fe7551e288f3>, line 1)

What if you had a function that uses `*args` and you really wanted to allow users to define arguments via the keyword method? Well, that's what keyword-only arguments are for.

In [9]:
def my_func(a, *args, b):
    """
    Function that can receive an arbitrary number of arguments and has a 
    keyword-only argument, b.
    """
    # a, b work the same.
    print(a, b)
    # all other arguments get compacted into the tuple called args.
    print(args)
    return None

my_func(1, 2, 3, 4, 5, 6, b=7)

1 7
(2, 3, 4, 5, 6)


Note that b is written after the `*args`, meaning that it can only be reached if you use the keyword method to assign the argument.

What if you wanted to allow your users to pass any number of arguments using the keyword method? You use the `**kwargs` parameter:

In [10]:
def my_func(a, **kwargs):
    """
    Function that can receive an arbitrary number of keyword arguments.
    """
    # a, b work the same.
    print(a)
    # all other keyword arguments get compacted into the dictionary called kwargs.
    print(kwargs)
    return None

my_func(1, b=7, c=3)

1
{'b': 7, 'c': 3}


Note that there is nothing stoppig you from having both `*args` and `*kwargs` in the same function.

In [11]:
def my_func(a, *args, **kwargs):
    """
    Function that can receive an arbitrary number of positional and keyword arguments.
    """
    # a, b work the same.
    print(a)
    # all other positional arguments get compacted into the tuple called args.
    print(args)
    # all other keyword arguments get compacted into the tuple called args.
    print(kwargs)
    return None

my_func(1, 2, 3, 4, b=7, c=3)

1
(2, 3, 4)
{'b': 7, 'c': 3}


## Default Arguments <a class="anchor" id="default-arguments"></a>

**Default arguments** are arguments that can be set for a function so that you don't have to. They're certainly useful; however, there's a non-obvious danger hidden within if we're not careful.

To define a default argument, just type an `=` and the default value you want to use for the argument.

In [12]:
# An example of a function with default arguments

# Define a do-nothing function to test default arguments
def my_func(a: 'int',
            b: 'positional with default value of 5'=5,
            *args,
            c: 'required keyword only',
            d: 'keyword only with default value'='holi'
           ):
    print(a, b, c, d)
    return None

print('Running my_func passing all arguments.')
my_func(3, 2, d=1, c=0)
print('')
print('Running my_func with default values')
my_func(3, c=2)

Running my_func passing all arguments.
3 2 0 1

Running my_func with default values
3 5 2 holi


We can access (and modify) defaults for positional arguments using the `__defaults__` property, which returns a tuple of right-aligned default values.

In [13]:
# Print the default 
my_func.__defaults__

(5,)

When we define default values, they get assigned at the moment of the function definition.

In [14]:
t = 10

def my_func(a, b=t):
    print(a, b)
    return None

print('Running my_func.')
my_func(1)
print('')

t=5
print('Running my_func after modifying t.')
my_func(1)

Running my_func.
1 10

Running my_func after modifying t.
1 10


**Unless you know what you're doing, avoid using mutables as default arguments.**

In [15]:
def my_func(a, b=[]):
    b.append(1)
    print(a, b)
    return None

print('First run of my_func.')
my_func(1)
print('')

print('Second run of my_func.')
my_func(1)

First run of my_func.
1 [1]

Second run of my_func.
1 [1, 1]


An idea for a solution is to use constants:

In [16]:
# An example of what not to do.
DEFAULT_VALUE = []

def my_func(a, b=DEFAULT_VALUE):
    b.append(1)
    print(a, b)
    return None

print('Value of the constant before running my_func')
print(DEFAULT_VALUE)

print('First run of my_func.')
my_func(1)
print('')

print('Value of the constant after running my_func')
print(DEFAULT_VALUE)

Value of the constant before running my_func
[]
First run of my_func.
1 [1]

Value of the constant after running my_func
[1]


The idea is good, however it doesn't take into account that when using mutables, the *same* memory adress is kept, so modifying "inside" the function modifies the "outside" values. The following implementation solves the problem:

In [17]:
import copy

# An example of what not to do.
DEFAULT_VALUE = []

# Default is set to None
def my_func(a, b=None):
    # Make a copy agnostic to the data structure .
    b = b or copy.copy(DEFAULT_VALUE)
    b.append(1)
    print(a, b)
    return None

print('Value of the constant before running my_func')
print(DEFAULT_VALUE)

print('First run of my_func.')
my_func(1)
print('')

print('Value of the constant after running my_func')
print(DEFAULT_VALUE)

Value of the constant before running my_func
[]
First run of my_func.
1 [1]

Value of the constant after running my_func
[]


That works fine... Wait, how does the function know the value of `DEFAULT_VALUE`?

The answer lies in something called **scopes**.

## Namespaces & Scopes <a class="anchor" id="scopes"></a>

Variables and values need to live somewhere so that Python can access them. The places they live are called **namespaces**.

A namespace is a mapping of variable names to memory addresses. Basically, whenever you create a variable, a space in memory gets reserved for the value. Then, a (key, value) pair gets added to the namespace, where the key is the name of the variable and the value is the memory address. There are 3 basic namespaces:

1. **Built-in**: Everything that gets instanced when you load up Python. It contains base functions (`sum`, `max`, etc.) values (`None`, `False`, etc.).
2. **Global**: Whatever is in the `main` module.
3. **Local**: What is inside a function.

They follow a hierarchical structure:

<img src="https://media.geeksforgeeks.org/wp-content/uploads/types_namespace-1.png">

In [18]:
# Since we defined my_func in the main module, it's conatined in the global namespace.

# my_func's representation in the namespace
globals()['my_func']

<function __main__.my_func(a, b=None)>

Built-in and Global namespaces stay active as long as Python is; whereas Local namespaces are ephemeral in the sense that the moment the function stops running, everything in it disappears.

A **scope** refers to the namespaces a certain part of the program has access to:

1. Fist, Python will look in the smallest namespace it has access to (local if within a function and global if in the `main` module).
2. If the variable name isn't in that namespace, it looks at the next larger namespace.
3. Keep going up in the namespaces until the Built-in is reached.
4. If the variable name isn't found in Built-in, raise an error.

In [19]:
# As we know, my_func is in globals so it can be found.
print(f"Is my_func in the global namespace: {'my_func' in globals().keys()}")
my_func

Is my_func in the global namespace: True


<function __main__.my_func(a, b=None)>

In [20]:
# As we know, max is in built-in namespace so it can be found.
print(f"Is max in the global namespace: {'max' in globals().keys()}")
print('Still it can be found because it is in the built-in namespace.')
max

Is max in the global namespace: False
Still it can be found because it is in the built-in namespace.


<function max>

In [21]:
# Variable a that is used in the function my_func is not in the built-in or global namespaces, so it can't be found outside of the function.
print('Variables in my_func:')
print(my_func.__code__.co_varnames)

a

Variables in my_func:
('a', 'b')


NameError: name 'a' is not defined

You can nest local scopes within each other.

In [22]:
d = 15
def my_nested_func(a):
    b = 3
    def my_inner_func(a):
        c=5
        # d is taken from global,
        # c is taken from local inside my_inner_func
        # b and a are taken from local in my_nested_func
        print(a, b, c, d)
        return None
    my_inner_func(a)
    return None
my_nested_func(2)

2 3 5 15


One of the cool-yet-dangerous features of scopes is that they allow us to do **aliasing**, i.e., renaming variables and functions in different places to do different things.

In [23]:
d = 15
print(f'Original value for d: {d}')

def my_nested_func():
    d = 5
    def my_inner_func():
        d = 3
        print(f'The value of d inside my_inner_func: {d}')
        return None
    print(f'The value of d inside my_nested_func: {d}')
    my_inner_func()
    return None
my_nested_func()

print(f'Value of d after running my_nested_func: {d}')

Original value for d: 15
The value of d inside my_nested_func: 5
The value of d inside my_inner_func: 3
Value of d after running my_nested_func: 15


When does the aliasing happen? We could track it with the print function!

In [24]:
d = 15

def my_nested_func():
    print(f'Value of d at checkpoint 1: {d}')
    d = 5
    print(f'Value of d at checkpoint 2: {d}')
    def my_inner_func():
        d = 3
        return None
    my_inner_func()
    return None

my_nested_func()

UnboundLocalError: local variable 'd' referenced before assignment

Wait, what?

Turns out that when Python builds the function, it looks at all the variables that get defined inside of it and makes them local and will *not* look at other scopes (that is, unless you force it to).

In [25]:
d = 15

print(f'Original value for d: {d}')

def my_nested_func():
    global d
    print(f'Value of d at checkpoint 1: {d}')
    d = 5
    print(f'Value of d at checkpoint 2: {d}')
    def my_inner_func():
        d = 3
        return None
    my_inner_func()
    return None

my_nested_func()
print(f'Value of d after running my_nested_func: {d}')

Original value for d: 15
Value of d at checkpoint 1: 15
Value of d at checkpoint 2: 5
Value of d after running my_nested_func: 5


Can I only force Python to look at values in the Global scope? What if I wanted to use the value of `d` that is local to `my_nested_func` inside `my_inner_func`?

You can, and things get weird.

## Closures <a class="anchor" id="closures"></a>

Consider the following function:

In [26]:
d = 15
print(f'Global value for d: {d}')

def outer():
    d = 5
    def inner(a):
        return d + a
    return inner

func = outer()
print(func(3))

print(f'Global value for d: {d}')

Global value for d: 15
8
Global value for d: 15


Note that:

1. Outer returns the `inner` function, as opposed to the evaluation of inner. Remember that for Python EVERYTHING is an object, including functions, so there are no issues, although the construction could seem a bit odd.
2. We were able to run the inner function, and the value it took for parameter `d` was 5 (the same that was defined in the local scope of outer).

What is going on here?

When we run the line 

``func = outer()``

Python:

1. Creates the local variable `d`.
2. Creates the function `inner` that takes a single parameter `a` and references the __local__ variable `d`.
3. Return the function `inner`.

When inner get's returned, it has to "remember" the variable `d`, so it "pulls it along" when it gets returned. The group (function, variable) forms a **closure** and the variable `d` is what is known as a **free variable**. This name comes from the fact that `d` isn't local to `inner`, nor is it Global or Built-in; it's something intermediate. This scope is known as **non-local**.

In [27]:
func.__closure__

(<cell at 0x7f86a0369df8: int object at 0xa68b40>,)

In [28]:
func.__code__.co_freevars 

('d',)

You can also "force" a variable to form a part of a closure even if defined locally within `my_inner_func`.

In [29]:
def outer():
    d = 5
    def inner(a):
        nonlocal d
        d = 3
        return d + a
    return inner

func = outer()
func.__code__.co_freevars 

('d',)

What can we do with closures? There are several interesting applications; for now, we'll focus on decorators.

## Decorators <a class="anchor" id="decorators"></a>

**Decorators** are wrappers around functions that allow you to modify their behaviour while keeping your code clean and readable.

There are two kinds of decorators:

1. **Simple decorators**.
2. **Parametrized decorators**.

We'll cover each in turn.

### Simple Decorators <a class="anchor" id="simple-decorators"></a>

We'll start by building a decorator that measures the time a function takes to run.

#### Timing a function

To time a function we'll use `perf_counter`, which gives us a the computer's time when called. Therefore, we can compute the elapsed time as follows:

In [30]:
from time import perf_counter

In [31]:
# Time before starting.
start = perf_counter()
# Run the function.
result = my_func(5)
# Time at end.
end = perf_counter()

# Compute elapsed time.
elapsed = end - start

print(f'Elapsed time: {elapsed:.6f}s.')

5 [1]
Elapsed time: 0.007463s.


#### Generalizing to a function

Now, we don't want to keep copying and pasting this code for every function we want to time would be unwieldy and innefficient, so why don't we generalize into a function?

In [32]:
def time_fn(fn, *args, **kwargs):
    """
    Time a function
    """
    # Time before starting.
    start = perf_counter()
    # Run the function.
    result = fn(*args, **kwargs)
    # Time at end.
    end = perf_counter()

    # Compute elapsed time.
    elapsed = end - start

    # Print elapsed time.
    print(f'{fn.__name__} took {elapsed:.6f}s to run.')

    # Return value of the function.
    return result

time_fn(my_func, 1)

1 [1]
my_func took 0.005072s to run.


#### Building a decorator

Now, what if we wanted to *extend* the functionality of `my_func`, instead of wrapping it around another function? An example for this is the tren_de_vivenda pipeline, where several functions require a validation of whether the input is null and change the behaviour of the function if it is.

Note that in order to extend the functionality, we need to build a function that returns functions, not just the results.

In [33]:
# Decorator.
def timed(fn):
    """
    Time function fn.
    """
    # Note that fn is a free variable for inner.
    def inner(*args, **kwargs):
        """
        Inner function that actually performs the timing.
        """
        # Time before starting.
        start = perf_counter()
        # Run the function.
        result = fn(*args, **kwargs)
        # Time at end.
        end = perf_counter()
        
        # Compute elapsed time.
        elapsed = end - start
        
        # Print elapsed time.
        print(f'{fn.__name__} took {elapsed:.6f}s to run.')
        
        # Return value of the function.
        return result
    
    # Return closure.
    return inner

This structure wraps the timing functionality around fn, allowing us to create a brand new function that incorporates them both in a single, callable function. To apply the decorator, or "decorate" a function, you can use one of two syntaxes:

In [34]:
def fib_loop(n: 'int') -> 'int':
    """
    Compute n-th Fibonnaci number using a loop.
    """
    fib_1 = 1
    fib_2 = 1
    for i in range(3, n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2 

fib_loop(90)

2880067194370816120

In [35]:
def fib_loop(n: 'int') -> 'int':
    """
    Compute n-th Fibonnaci number using a loop.
    """
    fib_1 = 1
    fib_2 = 1
    for i in range(3, n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2 

fib_loop = timed(fib_loop)
fib_loop(90)

fib_loop took 0.000044s to run.


2880067194370816120

In [36]:
@timed
def fib_loop(n: 'int') -> 'int':
    """
    Compute n-th Fibonnaci number using a loop.
    """
    fib_1 = 1
    fib_2 = 1
    for i in range(3, n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2 

fib_loop(90)

fib_loop took 0.000017s to run.


2880067194370816120

#### Further improvements

Up to here, everything looks fine, however, if we look at the documentation for `fib_loop`, we'll notice something strange.

In [37]:
print(fib_loop.__doc__)


        Inner function that actually performs the timing.
        


We have lost our original documentation!

In order to fix this, we could choose all the attributes that we want our function to retain and set them manually, or we can use some of Python's built-in goodness.

In [38]:
from functools import wraps

# Decorator.
def timed(fn):
    """
    Time function fn.
    """
    
    # Make the function keep its name and documentation.
    @wraps(fn)
    def inner(*args, **kwargs):
        """
        Inner function that actually performs timing.
        """
        # Time before starting.
        start = perf_counter()
        # Run the function.
        result = fn(*args, **kwargs)
        # Time at end.
        end = perf_counter()
        
        # Compute elapsed time.
        elapsed = end - start
        
        # Print elapsed time.
        print(f'{fn.__name__} took {elapsed:.6f}s to run.')
        
        # Return value of the function.
        return result
    
    # Return closure.
    return inner

In [39]:
@timed
def fib_loop(n: 'int') -> 'int':
    """
    Compute n-th Fibonnaci number using a loop,
    """
    fib_1 = 1
    fib_2 = 1
    for i in range(3, n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2 

fib_loop(90)

fib_loop took 0.000020s to run.


2880067194370816120

In [40]:
print(fib_loop.__doc__)


    Compute n-th Fibonnaci number using a loop,
    


We have managed to keep our documentation (and other attributes) thanks to `wraps`. Which uses the same syntax as a decorator and fullfills the same role as a decorator. Almost as if it were a decorator. Except it calls `fn`...

Well, if it looks like a decorator and works like a decorator, it must be a decorator, and unlike the `timed` decorator, it accepts parameters. This is why it's called a **parametrized decorator**.

### Parametrized Decorators <a class="anchor" id="parametrized-decorators"></a>

**Paramtrized decorators** are, as the name implies, decorators that accept parameters; and examples include the `wraps` decorator we saw previously, as well as Flask endpoint definitions:

```
@app.route('/healthcheck')
def healthcheck():
    return jsonify(status='ok')
```

If we think about it, a parametrized decorator is a decorator that changes its behaviour based on an additional parameter, meaning that it's something that must _create_ a decorator based on the value of the parameter. Therefore, a parametrized decorator must be a function that returns decorators!

To understand how they work, let's extend our `timed` decorator to make it run the function several times, where the exact number of times is a parameter.

#### First step: Figure out what the decorator should look like

We'll assume that the number of times is fixed, say, 10 to figure out how the returned decorator should be written.

In [41]:
# Decorator.
def timed(fn):
    """
    Time function fn.
    """
    
    # Make the function keep its name and documentation.
    @wraps(fn)
    def inner(*args, **kwargs):
        """
        Inner function that actually performs timing.
        """
        # Variable to keep track of time elapsed in repetition
        elapsed = 0

        # Number of iterations
        num_iters = 10
        
        # Iterate 10 times
        for i in range(num_iters):

            # Time before starting.
            start = perf_counter()
            # Run the function.
            result = fn(*args, **kwargs)
            # Time at end.
            end = perf_counter()

            # Compute elapsed time.
            elapsed += end - start
        
        # Print elapsed time.
        print(f'{fn.__name__} took an average of {elapsed/num_iters:.6f}s to run.')
        
        # Return value of the function.
        return result
    
    # Return closure.
    return inner

In [42]:
@timed
def fib_loop(n: 'int') -> 'int':
    """
    Compute n-th Fibonnaci number using a loop,
    """
    fib_1 = 1
    fib_2 = 1
    for i in range(3, n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2 

fib_loop(90)

fib_loop took an average of 0.000033s to run.


2880067194370816120

#### Step 2: Enclose it around a function that takes the parameter

In [43]:
# Parametrized decorator
def time_multiples(num_iters):
    """
    Time a function num_iters times.
    """
    
    # Decorator.
    def timed(fn):
        """
        Time function fn.
        """

        # Make the function keep its name and documentation.
        @wraps(fn)
        def inner(*args, **kwargs):
            """
            Inner function that actually performs timing.
            """
            # Variable to keep track of time elapsed in repetition
            elapsed = 0

            # Iterate 10 times
            for num_iter in range(num_iters):
                print(f'Running for the {num_iter+1}-th time.')
                # Time before starting.
                start = perf_counter()
                # Run the function.
                result = fn(*args, **kwargs)
                # Time at end.
                end = perf_counter()

                # Compute elapsed time.
                elapsed += end - start

            # Print elapsed time.
            print(f'{fn.__name__} took an average of {elapsed/num_iters:.6f}s to run.')

            # Return value of the function.
            return result

        # Return closure.
        return inner
    # Return decorator
    return timed

In [44]:
def fib_loop(n: 'int') -> 'int':
    """
    Compute n-th Fibonnaci number using a loop,
    """
    fib_1 = 1
    fib_2 = 1
    for i in range(3, n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2 

fib_loop_timed = time_multiples(5)(fib_loop)
fib_loop_timed(90)

Running for the 1-th time.
Running for the 2-th time.
Running for the 3-th time.
Running for the 4-th time.
Running for the 5-th time.
fib_loop took an average of 0.000020s to run.


2880067194370816120

In [45]:
fib_loop(90)

2880067194370816120

In [46]:
@time_multiples(5)
def fib_loop(n: 'int') -> 'int':
    """
    Compute n-th Fibonnaci number using a loop,
    """
    fib_1 = 1
    fib_2 = 1
    for i in range(3, n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2 

fib_loop(90)

Running for the 1-th time.
Running for the 2-th time.
Running for the 3-th time.
Running for the 4-th time.
Running for the 5-th time.
fib_loop took an average of 0.000016s to run.


2880067194370816120

## Concluding Thoughts <a class="anchor" id="conclusions"></a>

I believe that a profound understanding of the tools we use can help us become better programmers, write better code, and lay the foundations of what is to come. I hope all of you found somethig new and useful about Python in this Ignition. Thank you very much.

## References <a class="anchor" id="references"></a>

Most of this Ignition (including the examples of decorators and some of the code) is based on Fred Baptiste's course, [Python 3: Deep Dive (Part 1 - Functional)](https://www.udemy.com/course/python-3-deep-dive-part-1/) available in Udemy.

Another interesting reference for decorators is ["Closures and decorators" by Reza Bagheri](https://towardsdatascience.com/closures-and-decorators-in-python-2551abbc6eb6?gi=379f6abd1aaf).


Namespaces image taken from GeeksforGeeks, URL: https://media.geeksforgeeks.org/wp-content/uploads/types_namespace-1.png.